## 0. Overview – What this Python script does

This script builds a clean panel dataset for your FDI project. It:

* Loads the World Bank income classification file and keeps only Low income, Lower middle income, and Upper middle income countries.

* Reads all indicator files (FDI, GDP, trade, inflation, electricity, education, CPI, LPI).

* Converts wide datasets (years as columns) into long format
(Country, Code, Year, Value).

* Filters all data to 2010–2024.

* Merges everything into one panel file: fdi_master_panel.cleaned.csv.Import the relevant Python packages and use the income classification 


## 1. Import Libraries and Create DataProcessor Class

* This imports required libraries and creates the reusable DataProcessor class, which manages all loading, cleaning, reshaping, and merging tasks.

In [194]:
import pandas as pd
import re
from functools import reduce
import os



In [195]:
class FDIProcessor:
    def __init__(self, year_min, year_max, income_file):
        self.YEAR_MIN = year_min
        self.YEAR_MAX = year_max
        self.INCOME_FILE = income_file
        self.allowed_codes = set()
        self.income_filtered = pd.DataFrame()

## 2. Load Income Classification & Filter LIC/LMIC/UMIC

This function:

* Loads the income classification Excel file

* Standardizes columns to: Country, Code, Income_group

* Filters to only Low, Lower-Middle, and Upper-Middle Income countries

* Stores allowed ISO3 codes for later filtering.

In [197]:
    # 1. LOAD INCOME CLASSIFICATION
    def load_income_classification(self):
        income = pd.read_excel(self.INCOME_FILE)

        # Standardize column names
        col_map = {}
        for c in income.columns:
            cl = c.lower()
            if "code" in cl:
                col_map[c] = "Code"
            elif "income" in cl:
                col_map[c] = "Income_group"
            elif any(w in cl for w in ["country", "economy", "name"]):
                col_map[c] = "Country"

        income = income.rename(columns=col_map)

        allowed_groups = ["Low income", "Lower middle income", "Upper middle income"]
        self.income_filtered = income[income["Income_group"].isin(allowed_groups)].copy()
        self.allowed_codes = set(self.income_filtered["Code"].unique())

        print(f"\nAllowed LIC/LMIC/UMIC country codes found: {len(self.allowed_codes)}")


## 3. Helper: Standardize Country & Code Column Names

* Many datasets name country columns differently (e.g., Economy, Country Name, Nation).
* This helper ensures consistent names (Country, Code) everywhere.

In [199]:
    # 2. STANDARDIZE CODE/COUNTRY COLUMNS
    def _standardize_country_code_cols(self, df):
        c_map = {}
        for c in df.columns:
            cl = c.lower()
            if "code" in cl:
                c_map[c] = "Code"
            elif any(w in cl for w in ["country", "economy", "name"]):
                c_map[c] = "Country"

        return df.rename(columns=c_map)


## 4. Convert Wide Datasets to Long Format

* This converts files like GDP, Trade, Education, etc. 
from:
2010 | 2011 | 2012 | ...
to:
Country | Code | Year | Value

* This is essential for merging and analysis.

In [201]:
    # 3. GENERIC WIDE → LONG LOADER
    def wide_to_long_file(self, path, value_name, encoding=None):
        df = pd.read_csv(path, encoding=encoding)

        df = self._standardize_country_code_cols(df)
        df = df[df["Code"].isin(self.allowed_codes)].copy()

        id_vars = ["Country", "Code"]

        # Extract year columns using regex
        year_cols_map = {
            col: re.search(r'\d{4}', str(col)).group(0)
            for col in df.columns
            if col not in id_vars and re.search(r'\d{4}', str(col))
        }

        df = df[id_vars + list(year_cols_map.keys())]
        df = df.rename(columns=year_cols_map)
        year_cols = list(year_cols_map.values())

        long_df = df.melt(
            id_vars=id_vars,
            value_vars=year_cols,
            var_name="Year",
            value_name=value_name
        )

        long_df["Year"] = pd.to_numeric(long_df["Year"], errors="coerce").astype("Int64")
        long_df = long_df[
            (long_df["Year"] >= self.YEAR_MIN) & (long_df["Year"] <= self.YEAR_MAX)
        ]

        print(f"{os.path.basename(path):20s} → long shape: {long_df.shape}")
        return long_df


## 5. Special Handler for CPI (Already Long Format)

CPI (Corruption Perceptions Index) is already long format. This function: 

* Detects the CPI column

* Renames it to "CPI"

* Filters by country code and year range

In [203]:
    # 4. SPECIAL HANDLER FOR CPI (ALREADY LONG FORMAT)
    def load_cpi_long(self, path):
        cpi = pd.read_csv(path)
        cpi = self._standardize_country_code_cols(cpi)

        for c in cpi.columns:
            if "corruption" in c.lower():
                cpi = cpi.rename(columns={c: "CPI"})
                break

        cpi = cpi[["Country", "Code", "Year", "CPI"]]
        cpi = cpi[cpi["Code"].isin(self.allowed_codes)].copy()

        cpi["Year"] = pd.to_numeric(cpi["Year"], errors="coerce").astype("Int64")
        cpi = cpi[
            (cpi["Year"] >= self.YEAR_MIN) & (cpi["Year"] <= self.YEAR_MAX)
        ]

        print(f"{os.path.basename(path):20s} → CPI long shape: {cpi.shape}")
        return cpi



## 6. Process All Files with One Unified Function

This method loops over all datasets defined in FILE_MAP and processes each one using either:

* wide_to_long_file (default)

* load_cpi_long (for corruption dataset)

In [205]:
    # 5. PROCESS EACH DATASET IN FILE_MAP
    def process_all_files(self, file_map):
        dataframes = {}

        for name, details in file_map.items():
            path = details["path"]
            value_name = details["value_name"]
            encoding = details.get("encoding")
            handler_flag = details.get("handler", "wide")

            if handler_flag == "cpi":
                df = self.load_cpi_long(path)
            else:
                df = self.wide_to_long_file(path, value_name, encoding)

            if not df.empty:
                dataframes[name] = df

        return dataframes


## 7. Merge All Datasets Into One Panel

This inner-joins all long datasets on Country, Code, Year and attaches the Income_group.

In [207]:
    # 6. MERGE EVERYTHING INTO ONE PANEL
    def merge_all(self, dfs_to_merge):
        panel = reduce(
            lambda left, right: pd.merge(
                left, right, on=["Country", "Code", "Year"], how="inner"
            ),
            dfs_to_merge
        )

        panel = panel.merge(
            self.income_filtered[["Code", "Income_group"]],
            on="Code",
            how="left"
        )

        panel["Year"] = panel["Year"].astype(int)

        cols = ["Country", "Code", "Year", "Income_group"] + [
            c for c in panel.columns if c not in ["Country", "Code", "Year", "Income_group"]
        ]

        return panel[cols]


## 8. Configuration and Execution

This is the main execution block where:

* Years are defined (2010–2024)

* Dataset paths and variable names are set

* Processor runs income filtering, cleaning, long-format conversion, and merging

In [213]:
class FDIProcessor:
    def __init__(self, year_min, year_max, income_file):
        self.YEAR_MIN = year_min
        self.YEAR_MAX = year_max
        self.INCOME_FILE = income_file
        self.allowed_codes = set()
        self.income_filtered = pd.DataFrame()

    # 1. LOAD INCOME CLASSIFICATION (LIC/LMIC/UMIC)
    def load_income_classification(self):
        income = pd.read_excel(self.INCOME_FILE)

        # Standardize column names to: Country / Code / Income_group
        col_map = {}
        for c in income.columns:
            cl = c.lower()
            if "code" in cl:
                col_map[c] = "Code"
            elif "income" in cl:
                col_map[c] = "Income_group"
            elif any(w in cl for w in ["country", "economy", "name"]):
                col_map[c] = "Country"

        income = income.rename(columns=col_map)

        allowed_groups = ["Low income", "Lower middle income", "Upper middle income"]
        self.income_filtered = income[income["Income_group"].isin(allowed_groups)].copy()
        self.allowed_codes = set(self.income_filtered["Code"].unique())

        print(f"\nAllowed LIC/LMIC/UMIC country codes found: {len(self.allowed_codes)}")

    # 2. STANDARDIZE COUNTRY/CODE COLUMNS IN ANY DATAFRAME
    def _standardize_country_code_cols(self, df):
        c_map = {}
        for c in df.columns:
            cl = c.lower()
            if "code" in cl:
                c_map[c] = "Code"
            elif any(w in cl for w in ["country", "economy", "name"]):
                c_map[c] = "Country"
        return df.rename(columns=c_map)

    # 3. GENERIC WIDE → LONG LOADER (GDP, TRADE, INFLATION, etc.)
    def wide_to_long_file(self, path, value_name, encoding=None):
        df = pd.read_csv(path, encoding=encoding)

        df = self._standardize_country_code_cols(df)
        df = df[df["Code"].isin(self.allowed_codes)].copy()

        id_vars = ["Country", "Code"]

        # Detect year columns using regex
        year_cols_map = {
            col: re.search(r"\d{4}", str(col)).group(0)
            for col in df.columns
            if col not in id_vars and re.search(r"\d{4}", str(col))
        }

        df = df[id_vars + list(year_cols_map.keys())]
        df = df.rename(columns=year_cols_map)
        year_cols = list(year_cols_map.values())

        long_df = df.melt(
            id_vars=id_vars,
            value_vars=year_cols,
            var_name="Year",
            value_name=value_name,
        )

        long_df["Year"] = pd.to_numeric(long_df["Year"], errors="coerce").astype("Int64")
        long_df = long_df[
            (long_df["Year"] >= self.YEAR_MIN) & (long_df["Year"] <= self.YEAR_MAX)
        ]

        print(f"{os.path.basename(path):20s} → long shape: {long_df.shape}")
        return long_df

    # 4. SPECIAL HANDLER FOR CPI (ALREADY LONG FORMAT)
    def load_cpi_long(self, path):
        cpi = pd.read_csv(path)
        cpi = self._standardize_country_code_cols(cpi)

        # Find the CPI column by name
        for c in cpi.columns:
            if "corruption" in c.lower():
                cpi = cpi.rename(columns={c: "CPI"})
                break

        cpi = cpi[["Country", "Code", "Year", "CPI"]]
        cpi = cpi[cpi["Code"].isin(self.allowed_codes)].copy()

        cpi["Year"] = pd.to_numeric(cpi["Year"], errors="coerce").astype("Int64")
        cpi = cpi[
            (cpi["Year"] >= self.YEAR_MIN) & (cpi["Year"] <= self.YEAR_MAX)
        ]

        print(f"{os.path.basename(path):20s} → CPI long shape: {cpi.shape}")
        return cpi

    # 5. PROCESS ALL FILES ACCORDING TO FILE_MAP
    def process_all_files(self, file_map):
        dataframes = {}

        for name, details in file_map.items():
            path = details["path"]
            value_name = details["value_name"]
            encoding = details.get("encoding")
            handler_flag = details.get("handler", "wide")

            if handler_flag == "cpi":
                df = self.load_cpi_long(path)
            else:
                df = self.wide_to_long_file(path, value_name, encoding)

            if not df.empty:
                dataframes[name] = df

        return dataframes

    # 6. MERGE EVERYTHING INTO ONE PANEL
    def merge_all(self, dfs_to_merge):
        panel = reduce(
            lambda left, right: pd.merge(
                left, right, on=["Country", "Code", "Year"], how="inner"
            ),
            dfs_to_merge,
        )

        panel = panel.merge(
            self.income_filtered[["Code", "Income_group"]],
            on="Code",
            how="left",
        )

        panel["Year"] = panel["Year"].astype(int)

        cols = ["Country", "Code", "Year", "Income_group"] + [
            c for c in panel.columns if c not in ["Country", "Code", "Year", "Income_group"]
        ]

        panel = panel[cols]
        print("\nFinal merged panel shape:", panel.shape)
        return panel


In [223]:

year_min = 2010
year_max = 2024
income_file = "income_class.xlsx"

FILE_MAP = {
    "FDI":        {"path": "nations_netFDI.csv",    "value_name": "FDI_inflows"},
    "GDP":        {"path": "country_GDP_size.csv",  "value_name": "GDP_current_USD"},
    "GDP_growth": {"path": "annual_GDP_growth.csv", "value_name": "GDP_growth"},
    "Trade":      {"path": "trade_openness.csv",    "value_name": "Trade_pct_GDP"},
    "Inflation":  {"path": "inflation.csv",         "value_name": "Inflation_CPI"},
    "Electricity":{"path": "electricity.csv",       "value_name": "Electricity_access"},
    "Education":  {"path": "education.csv",         "value_name": "Education_enrollment"},
    "LPI":        {"path": "LPI_score.csv",         "value_name": "LPI_score", "encoding": "latin1"},
    "CPI":        {"path": "corruption.csv",        "value_name": "CPI", "handler": "cpi"},
}

processor = FDIProcessor(year_min, year_max, income_file)

processor.load_income_classification()

processed_dfs = processor.process_all_files(FILE_MAP)

dfs_list = list(processed_dfs.values())
final_panel = processor.merge_all(dfs_list)

final_panel.head()



Allowed LIC/LMIC/UMIC country codes found: 129
nations_netFDI.csv   → long shape: (1935, 4)
country_GDP_size.csv → long shape: (1935, 4)
annual_GDP_growth.csv → long shape: (1935, 4)
trade_openness.csv   → long shape: (1935, 4)
inflation.csv        → long shape: (1935, 4)
electricity.csv      → long shape: (1935, 4)
education.csv        → long shape: (1935, 4)
LPI_score.csv        → long shape: (570, 4)
corruption.csv       → CPI long shape: (1533, 4)

Final merged panel shape: (403, 13)


,Country,Code,Year,Income_group,FDI_inflows,GDP_current_USD,GDP_growth,Trade_pct_GDP,Inflation_CPI,Electricity_access,Education_enrollment,LPI_score,CPI
0,Afghanistan,AFG,2012,Low income,5.682366e+07,1.990733e+10,12.752287,NaN,6.441213,69.1,54.372860,2.30,8.0
1,Angola,AGO,2012,Lower middle income,-1.464628e+09,1.280000e+11,8.542107,91.800097,10.277905,37.3,NaN,2.28,22.0
2,Albania,ALB,2012,Upper middle income,9.179946e+08,1.224650e+10,0.984130,76.968358,2.031593,99.9,97.608612,2.77,33.0
3,Argentina,ARG,2012,Upper middle income,1.532393e+10,5.460000e+11,-1.026420,30.526542,NaN,99.1,102.709992,3.05,35.0
4,Armenia,ARM,2012,Upper middle income,4.966367e+08,1.061932e+10,7.200000,75.961676,2.558020,99.5,NaN,2.56,34.0


In [225]:
# === SAVE FULL LEFT JOIN PANEL ===
final_panel_left = final_panel.copy()
final_panel_left.to_csv("panel_full_left_join.csv", index=False)

print("\n✅ Saved: panel_full_left_join.csv (all years, NaNs allowed)")
print("Shape:", final_panel_left.shape)

# === SAVE INNER JOIN / COMPLETE-CASE PANEL ===
final_panel_inner = final_panel.dropna().copy()
final_panel_inner.to_csv("panel_regression_inner_join.csv", index=False)

print("\n✅ Saved: panel_regression_inner_join.csv (complete cases only)")
print("Shape:", final_panel_inner.shape)



✅ Saved: panel_full_left_join.csv (all years, NaNs allowed)
Shape: (403, 13)

✅ Saved: panel_regression_inner_join.csv (complete cases only)
Shape: (218, 13)
